In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import math
import matplotlib.pyplot as plt
import pymc as pm
from itertools import combinations
import arviz as az
import theano.tensor as tt
import nfl_data_py as nfl
import ast
from scipy.stats import norm

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
df_nfl_games=nfl.import_schedules([2024])

In [3]:
df_nfl_games["actual_total"]=df_nfl_games["home_score"]+df_nfl_games["away_score"]

df_nfl_games["home_score_reg"] = np.where(df_nfl_games.overtime==1,df_nfl_games[['home_score','away_score']].min(axis=1) , df_nfl_games["home_score"])
df_nfl_games["away_score_reg"] = np.where(df_nfl_games.overtime==1,df_nfl_games[['home_score','away_score']].min(axis=1) , df_nfl_games["away_score"])



In [4]:
teams = df_nfl_games["home_team"].unique()
teams = np.sort(teams)
teams = pd.DataFrame(teams, columns=['team'])
teams['i'] = teams.index

# Create a unique list of each team combination and assign an integer label
# to each one. Also decide which team will be 'heads' in each pair.
all_teams_pair_combinations = combinations(teams['team'], 2)
team_pairs_dict = {}
team_pairs_heads_dict = {}
pair_index = 0
for pair in all_teams_pair_combinations:
    team_pairs_dict[(pair[0], pair[1])] = pair_index
    team_pairs_dict[(pair[1], pair[0])] = pair_index
    team_pairs_heads_dict[(pair[0], pair[1])] = pair[0]
    team_pairs_heads_dict[(pair[1], pair[0])] = pair[0]
    pair_index += 1

num_teams = 32

def add_team_data_labels(game_data):
    game_data = game_data.merge(teams, left_on='home_team', right_on='team', how='left')
    game_data = game_data.rename(columns={'i': 'i_home'}).drop('team', axis=1)
    game_data = game_data.merge(teams, left_on='away_team', right_on='team', how='left')
    game_data = game_data.rename(columns={'i': 'i_away'}).drop('team', axis=1)
    game_data['i_pair'] = game_data.apply(lambda row: team_pairs_dict[(row['home_team'], row['away_team'])], axis=1)  
    
    return game_data

In [5]:
df_nfl_games=add_team_data_labels(df_nfl_games)

In [6]:
df_nfl_games[["home_team","away_team"]].to_clipboard()

In [7]:
df_team_priors = pd.read_csv("D:/NFL/Totals/nfl_2024_priors.csv").iloc[:,:]
df_team_priors.columns=["team", "off", "def"]
df_team_priors["def"]=-1*df_team_priors["def"]
df_team_priors.head(20)

,team,off,def
0,KC,4.00,0.52
1,PHI,2.70,0.32
2,ATL,1.60,0.02
3,BUF,2.40,0.50
4,CHI,0.30,0.22
5,CIN,1.70,-0.68
6,IND,-2.00,-0.78
7,MIA,1.70,1.02
8,NO,-1.45,-0.58
9,NYG,-2.40,-1.60


In [8]:
current_week=10

In [9]:
priors_df = pd.read_csv("D:/NFL/Totals/NFL_priors.csv").iloc[:,1:]

In [10]:
priors_df

,off_mu,off_sd,def_mu,def_sd,week
0,"[-0.9, 1.6, 2.5, 2.4, -3.0, 0.3, 1.7, -1.7, 2....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-1.4, 0.02, 0.62, 0.5, -0.48, 0.22, -0.68, 1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[-0.8346134938332402, 1.3829344114506126, 2.42...","[0.9683396241702018, 0.9855013150539457, 0.988...","[-1.4600746793077917, 0.0699238874377564, 0.65...","[0.9884734667769318, 0.9980178977679942, 0.999...",2
2,"[-0.6370139900926087, 1.4097573514779185, 2.40...","[0.9692370249984694, 0.9492733669784303, 0.983...","[-1.3001365853975366, 0.10178812121159876, 0.5...","[0.9854770182531544, 0.9826268380421239, 0.986...",3
3,"[-0.726945595554137, 1.3392539850798253, 2.479...","[0.9498829395378843, 0.9289123012197598, 0.991...","[-1.2623720459447016, 0.11689109705664957, 0.5...","[0.9707245879405095, 0.9587243135959667, 0.954...",4
4,"[-0.8267042241388167, 1.343584723476025, 2.590...","[0.9451496000218002, 0.9080032402691995, 0.965...","[-1.5172303881412446, 0.0651992393347912, 0.67...","[0.9456783476613236, 0.9513965955636479, 0.945...",5
5,"[-0.7982186557418116, 1.36211783651831, 2.6479...","[0.9288314730025855, 0.8940092328165302, 0.950...","[-1.4750090442579995, 0.028327728312311447, 0....","[0.9375998900648439, 0.9391131723849886, 0.911...",6
6,"[-0.8575791940590453, 1.427348806920136, 2.647...","[0.9320387237283193, 0.8937018621550102, 0.915...","[-1.4914359004915543, 0.058104390751295074, 0....","[0.9308848184034411, 0.9421213494533238, 0.899...",7
7,"[-0.8891004701058752, 1.388734122557431, 2.731...","[0.9076240683697883, 0.87441124777854, 0.89683...","[-1.4353616233611641, -0.009299206412093777, 0...","[0.9320668652234778, 0.9169714749695183, 0.891...",8
8,"[-0.8495502629882156, 1.399245388329618, 2.731...","[0.8745036419010243, 0.8527344418076863, 0.873...","[-1.451703350044664, -0.022379742709861292, 0....","[0.9389238564173142, 0.9206080899044693, 0.868...",9


In [11]:
# off_list=[]
# def_list=[]
# for team in teams.team:
#     off_list.append(df_team_priors[df_team_priors["team"]==team]["off"].iloc[0])
#     def_list.append(df_team_priors[df_team_priors["team"]==team]["def"].iloc[0])
    
    

In [12]:
df_nfl_games=df_nfl_games[["game_id", "game_type", "week", "home_team", "away_team", "home_score_reg", "away_score_reg", "total", "spread_line","total_line", "roof", "surface", "temp", "wind", "home_qb_name", "away_qb_name", "stadium", "i_home", "i_away", "i_pair"]]

In [13]:
pred_week=10
avg_pts = 21.9
home=1.8

In [14]:
def mle_fit(data):
            """Return the mean and standard deviation of the normal distribution fitting the data using MLE."""
            return norm.fit(data)

def model_week(df_week,priors_df):
    home_scores = np.array(df_week["home_score_reg"])
    away_scores = np.array(df_week["away_score_reg"])
    home_team=np.array(df_week["i_home"])
    away_team=np.array(df_week["i_away"])
    home_qb_adjust = np.array(df_week["qb_adjust_home"])
    away_qb_adjust = np.array(df_week["qb_adjust_away"])
    
    # qb_home = np.array(df_week["home_qb_value"].fillna(0))
    # qb_away = np.array(df_week["away_qb_value"].fillna(0))
    
    with pm.Model() as model:
        # Prior distributions for offensive and defensive strengths

        offence_star = pm.Normal('offence_star', mu=ast.literal_eval(priors_df["off_mu"].iloc[0]), sigma=ast.literal_eval(priors_df["off_sd"].iloc[0]), shape=32)
        defence_star = pm.Normal('defence_star', mu=ast.literal_eval(priors_df["def_mu"].iloc[0]), sigma=ast.literal_eval(priors_df["def_sd"].iloc[0]), shape=32)

        delta_o = pm.Normal('delta_o', mu=0.0, sigma=0.5, shape=32)
        delta_d = pm.Normal('delta_d', mu=0.0, sigma=0.5, shape=32)

        team_off_walked = pm.Deterministic('offence_walked', offence_star + delta_o)
        team_def_walked = pm.Deterministic('defence_walked', defence_star + delta_d)

        
        
        offence_mean = pm.math.sum(team_off_walked) / 32
        defence_mean = pm.math.sum(team_def_walked) / 32

      
        
        
         
        O = pm.Deterministic('offence', offence_star - offence_mean)
        D = pm.Deterministic('defence', defence_star - defence_mean)
    
        
    
        #
        # Expected points, now including HFA and mean_pts as variables
        Hp = O[home_team] - D[away_team] + home  + avg_pts
        Ap = O[away_team] - D[home_team] + avg_pts
    
        # Likelihood of observed data
        observed_home = pm.Normal('observed_home', mu=Hp, sigma=13, observed=home_scores+home_qb_adjust)
        observed_away = pm.Normal('observed_away', mu=Ap, sigma=13, observed=away_scores+away_qb_adjust)
    
        # Sample from the posterior
        trace = pm.sample(4000, tune=1500, cores=4,chains=1)

        offence_posterior_2024 = trace.posterior['offence'].values
        defence_posterior_2024 = trace.posterior['defence'].values
    
        
        offence_posterior_2024_mle = [mle_fit(offence_posterior_2024[..., i].flatten()) for i in range(32)]
        offence_posterior_2024_mu = [ele[0] for ele in offence_posterior_2024_mle]
        offence_posterior_2024_sd = [ele[1] for ele in offence_posterior_2024_mle]
        defence_posterior_2024_mle = [mle_fit(defence_posterior_2024[..., i].flatten()) for i in range(32)]
        defence_posterior_2024_mu = [ele[0] for ele in defence_posterior_2024_mle]
        defence_posterior_2024_sd = [ele[1] for ele in defence_posterior_2024_mle]

        poster_iter_dict={"offence_posterior_2024_mu":offence_posterior_2024_mu, "offence_posterior_2024_sd":offence_posterior_2024_sd,
                    "defence_posterior_2024_mu":defence_posterior_2024_mu, "defence_posterior_2024_sd":defence_posterior_2024_sd,
                   }

        return poster_iter_dict


        
        


In [15]:
pred_week_prev_df = df_nfl_games[df_nfl_games["week"]==pred_week-1].reset_index(drop=True).dropna(subset=["home_score_reg"]).reset_index(drop=True)
# pred_week_prev_df["qb_adjust_home"]=np.where(pred_week_prev_df["home_team"] =="GB",4.5,0)
# pred_week_prev_df["qb_adjust_away"]=np.where(pred_week_prev_df["away_team"] =="GB",4.5,0)
pred_week_prev_df["qb_adjust_home"]=np.where(pred_week_prev_df["home_team"] =="MIAZ",3.5,0)
pred_week_prev_df["qb_adjust_away"]=np.where(pred_week_prev_df["away_team"] =="MIAZ",3.5,0)

pred_week_df = df_nfl_games[df_nfl_games["week"]==pred_week].reset_index(drop=True)

priors = priors_df[priors_df["week"]==pred_week-1]


posteriors=model_week(pred_week_prev_df,priors)


def total_func_iter(team1,team2):
    team1_idx = teams[teams["team"]==team1]["i"].iloc[0]
    team2_idx = teams[teams["team"]==team2]["i"].iloc[0]

    home_off= posteriors["offence_posterior_2024_mu"][team1_idx]+home - posteriors["defence_posterior_2024_mu"][team2_idx]
    away_off= posteriors["offence_posterior_2024_mu"][team2_idx] - posteriors["defence_posterior_2024_mu"][team1_idx]

    return home_off+away_off + avg_pts + avg_pts


predicted_total_list=[]
for i,row in pred_week_df.iterrows():

    predicted_total_list.append(total_func_iter(row["home_team"], row["away_team"]))


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [offence_star, defence_star, delta_o, delta_d]


Sampling 1 chain for 1_500 tune and 4_000 draw iterations (1_500 + 4_000 draws total) took 11 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [16]:
pred_week_df["predicted_total"] = predicted_total_list


In [17]:
pred_week_df

,game_id,game_type,week,home_team,away_team,home_score_reg,away_score_reg,total,spread_line,total_line,...,surface,temp,wind,home_qb_name,away_qb_name,stadium,i_home,i_away,i_pair,predicted_total
0,2024_10_CIN_BAL,REG,10,BAL,CIN,NaN,NaN,NaN,6.0,52.5,...,grass,NaN,NaN,Lamar Jackson,Joe Burrow,M&T Bank Stadium,2,6,64,50.478008
1,2024_10_NYG_CAR,REG,10,CAR,NYG,NaN,NaN,NaN,-5.0,41.5,...,grass,NaN,NaN,Andy Dalton,Daniel Jones,Allianz Arena,4,23,136,42.316787
2,2024_10_NE_CHI,REG,10,CHI,NE,NaN,NaN,NaN,6.5,39.5,...,grass,NaN,NaN,Caleb Williams,Drake Maye,Soldier Field,5,21,160,41.997020
3,2024_10_BUF_IND,REG,10,IND,BUF,NaN,NaN,NaN,-4.5,47.0,...,fieldturf,NaN,NaN,Joe Flacco,Josh Allen,Lucas Oil Stadium,13,3,99,46.436937
4,2024_10_MIN_JAX,REG,10,JAX,MIN,NaN,NaN,NaN,-4.0,46.5,...,grass,NaN,NaN,Trevor Lawrence,Sam Darnold,TIAA Bank Stadium,14,20,348,44.191730
5,2024_10_DEN_KC,REG,10,KC,DEN,NaN,NaN,NaN,9.5,42.0,...,grass,NaN,NaN,Patrick Mahomes,Bo Nix,GEHA Field at Arrowhead Stadium,15,9,248,46.386391
6,2024_10_ATL_NO,REG,10,NO,ATL,NaN,NaN,NaN,-3.5,47.5,...,sportturf,NaN,NaN,Derek Carr,Kirk Cousins,Mercedes-Benz Superdome,22,1,51,46.432947
7,2024_10_SF_TB,REG,10,TB,SF,NaN,NaN,NaN,-6.5,48.0,...,grass,NaN,NaN,Baker Mayfield,Brock Purdy,Raymond James Stadium,29,28,490,46.132120
8,2024_10_PIT_WAS,REG,10,WAS,PIT,NaN,NaN,NaN,2.5,44.5,...,grass,NaN,NaN,Marcus Mariota,Russell Wilson,FedExField,31,26,485,41.890363
9,2024_10_TEN_LAC,REG,10,LAC,TEN,NaN,NaN,NaN,8.0,37.5,...,matrixturf,NaN,NaN,Justin Herbert,Mason Rudolph,SoFi Stadium,17,30,403,42.670631


In [18]:
# pred_week_df["qb_adjust_home"]=np.where(pred_week_df["home_team"] =="GB",0,0)
# pred_week_df["qb_adjust_away"]=np.where(pred_week_df["away_team"] =="GB",0,0)
pred_week_df["qb_adjust_home"]=np.where(pred_week_df["home_team"] =="MIAZ",3.5,0)
pred_week_df["qb_adjust_away"]=np.where(pred_week_df["away_team"] =="MIAZ",3.5,0)

In [19]:
pred_week_df["predicted_total"] = pred_week_df["predicted_total"] - pred_week_df["qb_adjust_home"]- pred_week_df["qb_adjust_away"]

In [20]:
totals_pred_output = pred_week_df[["game_id", "predicted_total","total_line"]]

In [21]:
totals_pred_output["game_id"] = totals_pred_output["game_id"].str.replace(r"(?<!C)_LA(?!C)", "_LAR", regex=True)
totals_pred_output["game_id"]=totals_pred_output["game_id"].str.replace("LV","OAK")

C:\Users\nnich\AppData\Local\Temp\ipykernel_15544\848458396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totals_pred_output["game_id"] = totals_pred_output["game_id"].str.replace(r"(?<!C)_LA(?!C)", "_LAR", regex=True)
C:\Users\nnich\AppData\Local\Temp\ipykernel_15544\848458396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totals_pred_output["game_id"]=totals_pred_output["game_id"].str.replace("LV","OAK")


In [22]:
totals_pred_output.to_csv("D:/NFL/Totals/NFL_predicted_totals.csv")

In [23]:
totals_pred_output

,game_id,predicted_total,total_line
0,2024_10_CIN_BAL,50.478008,52.5
1,2024_10_NYG_CAR,42.316787,41.5
2,2024_10_NE_CHI,41.997020,39.5
3,2024_10_BUF_IND,46.436937,47.0
4,2024_10_MIN_JAX,44.191730,46.5
5,2024_10_DEN_KC,46.386391,42.0
6,2024_10_ATL_NO,46.432947,47.5
7,2024_10_SF_TB,46.132120,48.0
8,2024_10_PIT_WAS,41.890363,44.5
9,2024_10_TEN_LAC,42.670631,37.5


In [24]:
priors_df=priors_df[["off_mu","off_sd", "def_mu", "def_sd", "week"]]

In [168]:
posteriors["week"]=9

In [169]:
temp=pd.DataFrame([posteriors])
temp.columns=priors_df.columns

In [170]:
df_priors=pd.concat([priors_df,temp],axis=0)

In [171]:
df_priors.to_csv("D:/NFL/Totals/NFL_priors.csv")

In [ ]:
totals_pred_output

In [54]:
teamH="CIN"

In [55]:
teamH_ind=teams[teams["team"]==teamH]["i"].iloc[0]
for week in df_priors["week"].values:
    try:
        print(ast.literal_eval(df_priors[df_priors["week"]==week]["off_mu"].iloc[0])[teamH_ind])
    except:
        print(df_priors[df_priors["week"]==week]["off_mu"].iloc[0][teamH_ind])
        
    

1.7
1.481643562376081
1.5246076273227143
1.6088898346335188
1.734303786817829


In [56]:
teamH_ind=teams[teams["team"]==teamH]["i"].iloc[0]
for week in df_priors["week"].values:
    try:
        print(ast.literal_eval(df_priors[df_priors["week"]==week]["def_mu"].iloc[0])[teamH_ind])
    except:
        print(df_priors[df_priors["week"]==week]["def_mu"].iloc[0][teamH_ind])
        

-0.68
-0.6131684115379444
-0.6046120204137765
-0.8215495207466155
-0.8663962878965974


In [35]:

teamH_ind=teams[teams["team"]==teamH]["i"].iloc[0]
ast.literal_eval(df_priors.iloc[:1,:]["off_mu"].iloc[0])[teamH_ind]

4.0

In [36]:

teamH_ind=teams[teams["team"]==teamH]["i"].iloc[0]
df_priors.iloc[-1:,:]["off_mu"].iloc[0][teamH_ind]

3.870616721622639

In [83]:
df_priors

,off_mu,off_sd,def_mu,def_sd,week
0,"[-0.9, 1.6, 2.5, 2.4, -3.0, 0.3, 1.7, -1.7, 2....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-1.4, 0.02, 0.62, 0.5, -0.48, 0.22, -0.68, 1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[-0.8346134938332402, 1.3829344114506126, 2.42...","[0.9683396241702018, 0.9855013150539457, 0.988...","[-1.4600746793077917, 0.0699238874377564, 0.65...","[0.9884734667769318, 0.9980178977679942, 0.999...",2
2,"[-0.6370139900926087, 1.4097573514779185, 2.40...","[0.9692370249984694, 0.9492733669784303, 0.983...","[-1.3001365853975366, 0.10178812121159876, 0.5...","[0.9854770182531544, 0.9826268380421239, 0.986...",3
3,"[-0.726945595554137, 1.3392539850798253, 2.479...","[0.9498829395378843, 0.9289123012197598, 0.991...","[-1.2623720459447016, 0.11689109705664957, 0.5...","[0.9707245879405095, 0.9587243135959667, 0.954...",4
4,"[-0.8267042241388167, 1.343584723476025, 2.590...","[0.9451496000218002, 0.9080032402691995, 0.965...","[-1.5172303881412446, 0.0651992393347912, 0.67...","[0.9456783476613236, 0.9513965955636479, 0.945...",5
5,"[-0.7982186557418116, 1.36211783651831, 2.6479...","[0.9288314730025855, 0.8940092328165302, 0.950...","[-1.4750090442579995, 0.028327728312311447, 0....","[0.9375998900648439, 0.9391131723849886, 0.911...",6
6,"[-0.8575791940590453, 1.427348806920136, 2.647...","[0.9320387237283193, 0.8937018621550102, 0.915...","[-1.4914359004915543, 0.058104390751295074, 0....","[0.9308848184034411, 0.9421213494533238, 0.899...",7
0,"[-0.8891004701058752, 1.388734122557431, 2.731...","[0.9076240683697883, 0.87441124777854, 0.89683...","[-1.4353616233611641, -0.009299206412093777, 0...","[0.9320668652234778, 0.9169714749695183, 0.891...",7


In [37]:

teamH_ind=teams[teams["team"]==teamH]["i"].iloc[0]
ast.literal_eval(df_priors.iloc[:1,:]["def_mu"].iloc[0])[teamH_ind]

0.52

In [530]:
teamH_ind=teams[teams["team"]==teamH]["i"].iloc[0]
df_priors.iloc[-1:,:]["def_mu"].iloc[0][teamH_ind]

-0.7468547726731697

In [531]:
df_nfl_games["actual_total"].mean()

KeyError: 'actual_total'

In [88]:
posteriors

{'offence_posterior_2024_mu': [-0.6253535266747143,
  1.3898684041001532,
  2.410426019432897,
  2.5678759100632105,
  -3.322089324258328,
  0.16074061668681705,
  1.5327811446531336,
  -1.7862625678427182,
  2.2878677713487536,
  -2.9331112406313373,
  2.5861295565933244,
  2.023762598893664,
  2.577786453230169,
  -2.092472789851118,
  -0.3575950769048552,
  3.9741665025431505,
  1.5502113105012902,
  -0.25528685926216355,
  -2.0514086883726628,
  1.4538086143792281,
  -1.0011900801983173,
  -3.3431868096073485,
  -0.879014561067163,
  -2.60961795406232,
  1.1755055416554205,
  2.7326936985340025,
  -2.1231641008683044,
  -1.4890743898282353,
  3.5835587369311934,
  -1.922006795273628,
  -2.9481660547974853,
  -2.524748652765559],
 'offence_posterior_2024_sd': [0.96989480377012,
  0.9760147978344008,
  0.98251124861496,
  0.9672538034761892,
  0.9472304540655289,
  0.9867791021961732,
  0.9819016191668948,
  0.9747728127056569,
  0.9634005691664738,
  0.9398292690892447,
  0.97361758

In [47]:
teams

,team,i
0,ARI,0
1,ATL,1
2,BAL,2
3,BUF,3
4,CAR,4
5,CHI,5
6,CIN,6
7,CLE,7
8,DAL,8
9,DEN,9


In [48]:
pred_week_prev_df

,game_id,game_type,week,home_team,away_team,home_score_reg,away_score_reg,total,spread_line,total_line,...,temp,wind,home_qb_name,away_qb_name,stadium,i_home,i_away,i_pair,qb_adjust_home,qb_adjust_away
0,2024_02_BUF_MIA,REG,2,MIA,BUF,10.0,31.0,41.0,2.5,48.5,...,86.0,3.0,Tua Tagovailoa,Josh Allen,Hard Rock Stadium,19,3,105,0.0,0.0
1,2024_02_LV_BAL,REG,2,BAL,LV,23.0,26.0,49.0,9.0,41.0,...,76.0,10.0,Lamar Jackson,Gardner Minshew,M&T Bank Stadium,2,18,76,0.0,0.0
2,2024_02_LAC_CAR,REG,2,CAR,LAC,3.0,26.0,29.0,-5.0,39.0,...,78.0,13.0,Bryce Young,Justin Herbert,Bank of America Stadium,4,17,130,0.0,0.0
3,2024_02_NO_DAL,REG,2,DAL,NO,19.0,44.0,63.0,6.5,47.0,...,NaN,NaN,Dak Prescott,Derek Carr,AT&T Stadium,8,22,233,0.0,0.0
4,2024_02_TB_DET,REG,2,DET,TB,16.0,20.0,36.0,7.5,51.5,...,NaN,NaN,Jared Goff,Baker Mayfield,Ford Field,10,29,283,0.0,0.0
5,2024_02_IND_GB,REG,2,GB,IND,16.0,10.0,26.0,-3.0,41.5,...,85.0,7.0,Malik Willis,Anthony Richardson,Lambeau Field,11,13,287,4.5,0.0
6,2024_02_CLE_JAX,REG,2,JAX,CLE,13.0,18.0,31.0,3.0,41.5,...,80.0,12.0,Trevor Lawrence,Deshaun Watson,TIAA Bank Stadium,14,7,202,0.0,0.0
7,2024_02_SF_MIN,REG,2,MIN,SF,23.0,17.0,40.0,-4.5,46.5,...,NaN,NaN,Sam Darnold,Brock Purdy,U.S. Bank Stadium,20,28,437,0.0,0.0
8,2024_02_SEA_NE,REG,2,NE,SEA,20.0,20.0,43.0,-3.5,39.0,...,77.0,5.0,Jacoby Brissett,Geno Smith,Gillette Stadium,21,27,446,0.0,0.0
9,2024_02_NYJ_TEN,REG,2,TEN,NYJ,17.0,24.0,41.0,-3.5,41.5,...,80.0,13.0,Will Levis,Aaron Rodgers,Nissan Stadium,30,24,473,0.0,0.0
